# Load Packages

In [1]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.models.rue.save_load_model import load_model
from src.models.infernoise.tuning import tune_infernoise
from src.models.infernoise.predicting import infernoise_test_predictions
from src.misc import create_folder
# from seed_file import seed

seed = 2023
tuning_seed = 2023
data_label = "mimic"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_processed_folder = join(fp_project_folder, "processed_data", data_label)
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")

# Seed filepaths
fp_cur_tune_folder = join(fp_tuning, str(tuning_seed))
create_folder(fp_cur_tune_folder)
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)

2025-06-05 14:46:09.403012: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-05 14:46:09.414501: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-05 14:46:09.418032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-05 14:46:09.427608: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-05 14:46:10.049164: W tensorflow/compiler/tf2

# Load Data

In [2]:
split_dict = joblib.load(join(fp_processed_folder, "mimic_split_dict.joblib"))
predictors = ['ABPdias (mmHg)_mean Min0', 'RESP (bpm)_mean Min0', 'ABPmean (mmHg)_mean Min0', 'ABPsys (mmHg)_mean Min0', 'SpO2 (%)_mean Min0', 'HR (bpm)_mean Min0', 'ABPdias (mmHg)_std Min0', 'RESP (bpm)_std Min0', 'ABPmean (mmHg)_std Min0', 'ABPsys (mmHg)_std Min0', 'SpO2 (%)_std Min0', 'HR (bpm)_std Min0', 'ABPdias (mmHg)_mean Min1', 'RESP (bpm)_mean Min1', 'ABPmean (mmHg)_mean Min1', 'ABPsys (mmHg)_mean Min1', 'SpO2 (%)_mean Min1', 'HR (bpm)_mean Min1', 'ABPdias (mmHg)_std Min1', 'RESP (bpm)_std Min1', 'ABPmean (mmHg)_std Min1', 'ABPsys (mmHg)_std Min1', 'SpO2 (%)_std Min1', 'HR (bpm)_std Min1', 'ABPdias (mmHg)_mean Min2', 'RESP (bpm)_mean Min2', 'ABPmean (mmHg)_mean Min2', 'ABPsys (mmHg)_mean Min2', 'SpO2 (%)_mean Min2', 'HR (bpm)_mean Min2', 'ABPdias (mmHg)_std Min2', 'RESP (bpm)_std Min2', 'ABPmean (mmHg)_std Min2', 'ABPsys (mmHg)_std Min2', 'SpO2 (%)_std Min2', 'HR (bpm)_std Min2', 'ABPdias (mmHg)_mean Min3', 'RESP (bpm)_mean Min3', 'ABPmean (mmHg)_mean Min3', 'ABPsys (mmHg)_mean Min3', 'SpO2 (%)_mean Min3', 'HR (bpm)_mean Min3', 'ABPdias (mmHg)_std Min3', 'RESP (bpm)_std Min3', 'ABPmean (mmHg)_std Min3', 'ABPsys (mmHg)_std Min3', 'SpO2 (%)_std Min3', 'HR (bpm)_std Min3', 'ABPdias (mmHg)_mean Min4', 'RESP (bpm)_mean Min4', 'ABPmean (mmHg)_mean Min4', 'ABPsys (mmHg)_mean Min4', 'SpO2 (%)_mean Min4', 'HR (bpm)_mean Min4', 'ABPdias (mmHg)_std Min4', 'RESP (bpm)_std Min4', 'ABPmean (mmHg)_std Min4', 'ABPsys (mmHg)_std Min4', 'SpO2 (%)_std Min4', 'HR (bpm)_std Min4']

# Tuning

In [ ]:
if seed == tuning_seed:
    infernoise_hp_dict = {}
for time_label, info_dict in split_dict.items():
    print(f"{time_label}:")
    ae_regressor = load_model(name=f"rue_{time_label}", fp_checkpoints=fp_cur_model_folder)
    infernoise_tuning_df = tune_infernoise(
        ae_regressor, stddev_list=[0.00001, 0.00005, 0.0001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 
        valid_df=info_dict["valid_df"], inputs=predictors, outputs=info_dict["outputs"], seed=seed, T=10, regressor_label="_"+time_label
    )
    display(infernoise_tuning_df)
    infernoise_hp_dict[time_label] = infernoise_tuning_df.iloc[infernoise_tuning_df["loss"].argmin(), 0]

# Prediction

In [ ]:
infernoise_best_hp_dict = {'t+1': 1e-05, 't+2': 0.005, 't+3': 0.005}
for time_label, info_dict in split_dict.items():
    print(f"{time_label}:")
    ae_regressor = load_model(name=f"rue_{time_label}", fp_checkpoints=fp_cur_model_folder)
    infernoise_valid_df = infernoise_test_predictions(
         ae_regressor, test_df=info_dict["valid_df"], inputs=predictors, outputs=info_dict["outputs"], regressor_label="_"+time_label, 
        seed=seed, T=10, stddev=infernoise_best_hp_dict[time_label])
    infernoise_test_df = infernoise_test_predictions(
         ae_regressor, test_df=info_dict["test_df"], inputs=predictors, outputs=info_dict["outputs"], regressor_label="_"+time_label, 
        seed=seed, T=10, stddev=infernoise_best_hp_dict[time_label])
    display(infernoise_test_df)
    infernoise_valid_df.to_csv(join(fp_cur_predictions_folder, f"infernoise_valid_{time_label[-1]}.csv"))
    infernoise_test_df.to_csv(join(fp_cur_predictions_folder, f"infernoise_test_{time_label[-1]}.csv"))